# Memory Management

## Task of Memory Management

## Physical Mode

## Notes

Refer [here](https://jappavoo.github.io/UndertheCovers/textbook/unix/shellintro.html#arithmetic-expansion) for doing binary math in shell 